# Notebook 6 – Mini Sistema RAG Avanzado (sin transformers)

**Objetivo de este notebook:**  
Construir un **mini sistema RAG (Retrieval-Augmented Generation)** más cercano a lo que se usa en producción, pero usando solo herramientas livianas de Python:

1. **Documentos institucionales sintéticos** (inspirados en políticas y guías técnicas de SISRED/BCN).
2. **Chunking avanzado**:
   - División en fragmentos de tamaño controlado.
   - Ventanas solapadas para mejorar el contexto.
3. **Embeddings**:
   - TF-IDF sobre chunks.
   - Reducción de dimensionalidad con **LSA (TruncatedSVD)** para obtener vectores densos.
4. **Índice vectorial tipo FAISS/Chroma**:
   - `NearestNeighbors` con distancia coseno sobre los embeddings densos.
   - Re-ranqueo con **MMR (Maximal Marginal Relevance)** para evitar respuestas redundantes.
5. **Pipeline RAG simplificado**:
   - Recuperar los chunks más relevantes para una pregunta.
   - Construir un contexto consolidado.
   - Generar una **respuesta pseudo-resumida** sin LLM (para entender la mecánica).
6. **Mini-desafío**:
   - Usar el sistema para responder preguntas sobre documentos institucionales y analizar la calidad de la recuperación.

> En un sistema real, la parte de generación final (la “G” de RAG) la haría un LLM (p. ej. GPT), pero aquí vamos a abrir la “caja negra” del **Retrieval** y de cómo se arma el contexto.


In [14]:
# ============================================================
# Celda 2 – Imports principales (livianos, sin transformers)
# ============================================================

import re
import textwrap

import numpy as np
import pandas as pd

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import normalize
from sklearn.neighbors import NearestNeighbors


## 1. Dataset textual sintético (documentos SISRED/BCN)

Vamos a crear un **conjunto de documentos sintéticos** que simulan:

- Políticas internas de SISRED (seguridad, gestión de accesos).
- Guías de despliegue y operación.
- Procedimientos de gestión de incidentes.
- Buenas prácticas de desarrollo y documentación.
- Lineamientos para proyectos de IA y uso de modelos generativos.

En un sistema real, estos documentos podrían venir de:

- PDFs institucionales,
- wikis internas,
- repositorios de código con READMEs,
- manuales de operación,
- documentación de APIs y microservicios.

Trabajaremos con un `DataFrame` con columnas:

- `doc_id`: identificador del documento.
- `title`: título del documento.
- `text`: contenido completo del documento (texto largo).
- `category`: tipo de documento (seguridad, despliegue, IA, etc.) – útil como metadato.


In [15]:
# ============================================================
# Celda 4 – Crear documentos sintéticos enriquecidos
# ============================================================

docs_data = [
    {
        "doc_id": 1,
        "title": "Política de Seguridad de SISRED",
        "category": "seguridad",
        "text": """
        La política de seguridad de SISRED establece lineamientos obligatorios para el manejo
        de credenciales, control de accesos y protección de datos sensibles de los usuarios
        internos y externos. Todo acceso a sistemas de producción debe realizarse mediante
        autenticación de dos factores (2FA) y conexiones seguras (HTTPS o VPN corporativa).
        Se prohíbe explícitamente compartir contraseñas o tokens de acceso entre miembros
        del equipo. 

        Los perfiles de usuario deben implementarse bajo el principio de mínimo privilegio,
        revisándose al menos una vez al año. Cualquier incidente de seguridad, como accesos
        no autorizados, filtración de datos o comportamiento anómalo, debe reportarse al
        equipo de seguridad en un plazo máximo de 2 horas desde su detección, usando el
        canal oficial definido en la intranet de SISRED.
        """
    },
    {
        "doc_id": 2,
        "title": "Guía de Despliegue en Producción",
        "category": "despliegue",
        "text": """
        El despliegue de nuevas versiones de SISRED en producción debe seguir un flujo de
        integración continua. Cada cambio debe pasar por revisión de código (code review),
        pruebas automatizadas (unitarias y de integración) y un ambiente de staging que
        replique las condiciones de producción.

        Los despliegues se realizan preferentemente fuera del horario peak de uso, salvo
        incidentes críticos. Antes de desplegar, el responsable debe verificar que existe
        un plan de rollback documentado y probado, así como un checklist de validación
        funcional post-despliegue.

        Los logs de despliegue y monitoreo se almacenan al menos por 6 meses para fines
        de auditoría y análisis de incidentes. Todo despliegue debe quedar trazado con
        un identificador de versión (tag) y el responsable técnico asociado.
        """
    },
    {
        "doc_id": 3,
        "title": "Procedimiento de Gestión de Incidentes",
        "category": "operaciones",
        "text": """
        La gestión de incidentes en SISRED se divide en cuatro etapas: detección, análisis,
        mitigación y cierre. Los incidentes se clasifican según su impacto (alto, medio,
        bajo) y urgencia (crítico, urgente, normal). Una vez detectado un incidente, se debe
        registrar en la herramienta de seguimiento correspondiente.

        Durante la fase de mitigación es fundamental mantener informados a los usuarios
        afectados, utilizando canales como correo institucional o paneles de estado
        en la plataforma. Finalizada la mitigación, se ejecuta el análisis de causa raíz
        (RCA) y se definen acciones preventivas.

        El cierre formal del incidente requiere documentar el resumen del impacto, la causa
        raíz y las lecciones aprendidas. Esta información se almacena en el repositorio de
        documentación interna para consulta futura y auditorías.
        """
    },
    {
        "doc_id": 4,
        "title": "Buenas Prácticas de Desarrollo en SISRED",
        "category": "desarrollo",
        "text": """
        El desarrollo de nuevas funcionalidades para SISRED debe respetar buenas prácticas
        de ingeniería de software. Se recomienda aplicar principios SOLID, patrones de diseño
        apropiados y revisiones de código entre pares.

        Todas las funcionalidades deben incluir pruebas unitarias con cobertura mínima definida
        por el equipo, así como métricas de rendimiento para consultas a base de datos. Se debe
        evitar la introducción de dependencias innecesarias o librerías sin mantenimiento activo.

        Además, es obligatorio mantener documentación actualizada de APIs y contratos de datos,
        de modo que otros equipos puedan integrar sus servicios de manera segura y confiable.
        """
    },
    {
        "doc_id": 5,
        "title": "Lineamientos para Proyectos de IA en SISRED",
        "category": "ia",
        "text": """
        Los proyectos de Inteligencia Artificial en SISRED deben alinearse con los objetivos
        institucionales y respetar principios de transparencia y trazabilidad. Antes de iniciar
        un proyecto de IA, se debe definir claramente el problema, las métricas de éxito y las
        restricciones de uso de datos.

        La preparación de datos debe incluir anonimización cuando corresponda, gestión de sesgos
        y evaluación de la calidad de las fuentes. Los modelos de IA deben ser monitoreados en
        producción, registrando su desempeño y detectando posibles desviaciones o degradación
        en el tiempo.

        Para modelos de IA generativa, se debe documentar el contexto de uso, límites de decisión
        y mecanismos de supervisión humana. Toda integración de IA con SISRED debe pasar por
        revisión de seguridad, cumplimiento normativo y pruebas de robustez.
        """
    }
]

docs_df = pd.DataFrame(docs_data)
docs_df


,doc_id,title,category,text
0,1,Política de Seguridad de SISRED,seguridad,\n La política de seguridad de SISRED e...
1,2,Guía de Despliegue en Producción,despliegue,\n El despliegue de nuevas versiones de...
2,3,Procedimiento de Gestión de Incidentes,operaciones,\n La gestión de incidentes en SISRED s...
3,4,Buenas Prácticas de Desarrollo en SISRED,desarrollo,\n El desarrollo de nuevas funcionalida...
4,5,Lineamientos para Proyectos de IA en SISRED,ia,\n Los proyectos de Inteligencia Artifi...


In [16]:
# ============================================================
# Celda 5 – Exploración rápida del dataset
# ============================================================

print("Número de documentos:", len(docs_df))
docs_df[["doc_id", "title", "category"]]


Número de documentos: 5


,doc_id,title,category
0,1,Política de Seguridad de SISRED,seguridad
1,2,Guía de Despliegue en Producción,despliegue
2,3,Procedimiento de Gestión de Incidentes,operaciones
3,4,Buenas Prácticas de Desarrollo en SISRED,desarrollo
4,5,Lineamientos para Proyectos de IA en SISRED,ia


## 2. Chunking documental con ventanas solapadas

Para un sistema RAG, trabajar con documentos completos suele ser poco eficiente:  
los textos son largos y el modelo (o el vectorizador) pierde foco.

Aquí vamos a aplicar **chunking avanzado**:

1. Dividir cada documento en **pseudo-oraciones** usando puntos (`.`) y saltos de línea.
2. Construir **ventanas solapadas** de oraciones:
   - Por ejemplo, ventanas de 3 oraciones, con un solapamiento de 1.
   - Esto permite que un mismo concepto pueda aparecer en más de un chunk.
3. Crear una tabla de chunks con:
   - `chunk_id`
   - `doc_id`
   - `title`
   - `category`
   - `chunk_index` (posición del chunk dentro del documento)
   - `text_chunk`

💡 En producción, podríamos chunkear por:
- tokens (usando un tokenizer),
- párrafos,
- secciones con encabezados,
- o mezclas (tamaño + semántica).


In [17]:
# ============================================================
# Celda 7 – Chunking de documentos con ventanas solapadas
# ============================================================

def clean_text(text: str) -> str:
    """
    Limpieza ligera de texto:
    - Quita espacios duplicados.
    - Normaliza saltos de línea.
    """
    text = text.replace("\n", " ")
    text = re.sub(r"\s+", " ", text)
    return text.strip()


def split_into_sentences(text: str) -> list:
    """
    Divide en 'pseudo-oraciones' usando puntos como delimitador.
    Es una aproximación simple, suficiente para este ejercicio.
    """
    text = clean_text(text)
    # Dividir por punto, pero manteniendo algo de limpieza
    raw_sentences = re.split(r"\.\s+", text)
    sentences = [s.strip() for s in raw_sentences if s.strip()]
    return sentences


def sliding_window_chunks(sentences, window_size=3, overlap=1):
    """
    Genera ventanas solapadas de oraciones.
    window_size: cuántas oraciones por chunk.
    overlap: cuántas oraciones se reutilizan entre una ventana y la siguiente.
    """
    if window_size <= 0:
        raise ValueError("window_size debe ser > 0")
    if overlap >= window_size:
        raise ValueError("overlap debe ser menor que window_size")

    step = window_size - overlap
    chunks = []
    for start in range(0, len(sentences), step):
        window = sentences[start:start + window_size]
        if not window:
            break
        chunk_text = ". ".join(window)
        if not chunk_text.endswith("."):
            chunk_text += "."
        chunks.append(chunk_text)
        if start + window_size >= len(sentences):
            break
    return chunks


# Construir DataFrame de chunks
chunks = []
for _, row in docs_df.iterrows():
    doc_id = row["doc_id"]
    title = row["title"]
    category = row["category"]
    sentences = split_into_sentences(row["text"])

    text_chunks = sliding_window_chunks(
        sentences,
        window_size=3,
        overlap=1
    )

    for idx, ch in enumerate(text_chunks):
        chunks.append({
            "doc_id": doc_id,
            "title": title,
            "category": category,
            "chunk_index": idx,
            "chunk_id": f"{doc_id}_{idx}",
            "text_chunk": ch
        })

chunks_df = pd.DataFrame(chunks)
chunks_df.head()


,doc_id,title,category,chunk_index,chunk_id,text_chunk
0,1,Política de Seguridad de SISRED,seguridad,0,1_0,La política de seguridad de SISRED establece l...
1,1,Política de Seguridad de SISRED,seguridad,1,1_1,Se prohíbe explícitamente compartir contraseña...
2,2,Guía de Despliegue en Producción,despliegue,0,2_0,El despliegue de nuevas versiones de SISRED en...
3,2,Guía de Despliegue en Producción,despliegue,1,2_1,Los despliegues se realizan preferentemente fu...
4,2,Guía de Despliegue en Producción,despliegue,2,2_2,Los logs de despliegue y monitoreo se almacena...


In [18]:
# ============================================================
# Celda 8 – Resumen de chunks
# ============================================================

print("Total de chunks generados:", len(chunks_df))
chunks_df.groupby("doc_id")["chunk_id"].count().rename("num_chunks_por_doc")


Total de chunks generados: 13


doc_id
1    2
2    3
3    3
4    2
5    3
Name: num_chunks_por_doc, dtype: int64

## 3. Crear "embeddings" con TF-IDF + LSA (TruncatedSVD)

En sistemas RAG reales, se usan **embeddings densos** aprendidos por modelos de lenguaje
(`sentence-transformers`, `OpenAI embeddings`, etc.).  

Aquí vamos a construir algo similar, pero 100% local y liviano:

1. Representamos cada `text_chunk` con TF-IDF (`TfidfVectorizer`).
2. Aplicamos **TruncatedSVD** para reducir dimensionalidad:
   - Esto es equivalente a un modelo **LSA (Latent Semantic Analysis)**.
   - Obtenemos vectores densos (por ejemplo, de dimensión 64).
3. Normalizamos los vectores resultantes para poder usar **distancia coseno**.

Estos vectores densos serán nuestros **embeddings** para alimentar el índice tipo FAISS/Chroma.


In [21]:
# ============================================================
# Celda 10 – TF-IDF + LSA (TruncatedSVD)
# ============================================================
# Vectorizador TF-IDF

spanish_stopwords = ["de", "la", "que", "el", "en", "y", "a", "los", "del", "se", "las", "por", "un", 
                    "para", "con", "no", "una", "su", "al", "lo", "como", "más", "pero", "sus", "le", 
                    "ya", "o", "este", "sí", "porque", "esta", "entre", "cuando", "muy", "sin", "sobre", 
                    "también", "me", "hasta", "hay", "donde", "quien", "desde", "todo", "nos", "durante", 
                    "todos", "uno", "les", "ni", "contra", "otros", "ese", "eso", "ante", "ellos", "e", "esto", 
                    "mí", "antes", "algunos", "qué", "unos", "yo", "otro", "otras", "otra", "él", "tanto", "esa",
                    "estos", "mucho", "quienes", "nada", "muchos", "cual", "poco", "ella", "estaré"]

vectorizer = TfidfVectorizer(
    stop_words=spanish_stopwords,  # usar lista personalizada de stopwords en español (variable existente)
    ngram_range=(1, 2),            # unigramas y bigramas
    max_df=0.9,                    # ignorar términos muy frecuentes
    min_df=1                       # podríamos subir esto si hubiera muchos docs
)

X_tfidf = vectorizer.fit_transform(chunks_df["text_chunk"])
print("Shape matriz TF-IDF:", X_tfidf.shape)

# Reducir dimensionalidad con TruncatedSVD (LSA)
n_components = 64 if X_tfidf.shape[1] > 64 else min(32, X_tfidf.shape[1])
svd = TruncatedSVD(n_components=n_components, random_state=42)
X_lsa = svd.fit_transform(X_tfidf)

# Normalizar para que la distancia Euclidiana sea ~ distancia coseno
X_embeddings = normalize(X_lsa)

print("Shape matriz embeddings LSA:", X_embeddings.shape)
print("Varianza explicada acumulada (aprox):", svd.explained_variance_ratio_.sum())


Shape matriz TF-IDF: (13, 607)
Shape matriz embeddings LSA: (13, 13)
Varianza explicada acumulada (aprox): 0.9999999999999999


## 4. Índice vectorial tipo FAISS/Chroma

Con los embeddings densos listos, construimos un índice de búsqueda:

- Usaremos `NearestNeighbors` de `scikit-learn` con métrica **cosine**.
- Conceptualmente imita herramientas como **FAISS** o **Chroma**:
  - Guarda la matriz de embeddings.
  - Permite buscar los vectores más cercanos a una consulta.

Luego, para una `query`:

1. Vectorizamos la pregunta (TF-IDF + SVD + normalización).
2. Buscamos los chunks más cercanos en el índice.
3. Opcionalmente aplicamos **MMR (Maximal Marginal Relevance)** para evitar repetir chunks muy similares.


In [22]:
# ============================================================
# Celda 12 – Índice tipo vector DB (NearestNeighbors)
# ============================================================

nn_index = NearestNeighbors(
    n_neighbors=10,   # recuperamos más de lo que mostraremos, para aplicar MMR
    metric="cosine"
)

nn_index.fit(X_embeddings)


NearestNeighbors(metric='cosine', n_neighbors=10)

## 5. Función de búsqueda (retrieval) con MMR

Definiremos una función:

```python
retrieve_chunks(query, k=3, use_mmr=True)


In [23]:
# ============================================================
# Celda 14 – MMR y función de búsqueda
# ============================================================

def embed_query(query: str) -> np.ndarray:
    """
    Aplica el mismo pipeline de embeddings para una query:
    TF-IDF -> SVD -> normalización.
    """
    q_tfidf = vectorizer.transform([query])
    q_lsa = svd.transform(q_tfidf)
    q_emb = normalize(q_lsa)
    return q_emb


def maximal_marginal_relevance(query_vec, candidate_vecs, lambda_param=0.7, k=3):
    """
    Implementación simple de MMR.
    query_vec: vector (1, d)
    candidate_vecs: matriz (n_candidates, d)
    """
    # Similitud entre query y candidatos
    sim_to_query = (candidate_vecs @ query_vec.T).ravel()  # coseno aprox porque están normalizados

    # Matriz de similitud entre candidatos
    sim_between_candidates = candidate_vecs @ candidate_vecs.T

    selected_indices = []
    candidate_indices = list(range(candidate_vecs.shape[0]))

    for _ in range(min(k, candidate_vecs.shape[0])):
        mmr_scores = []
        for idx in candidate_indices:
            if not selected_indices:
                diversity_penalty = 0.0
            else:
                diversity_penalty = max(sim_between_candidates[idx, s] for s in selected_indices)
            mmr_score = lambda_param * sim_to_query[idx] - (1 - lambda_param) * diversity_penalty
            mmr_scores.append((idx, mmr_score))

        # Elegir el candidato con mayor MMR
        best_idx, _ = max(mmr_scores, key=lambda x: x[1])
        selected_indices.append(best_idx)
        candidate_indices.remove(best_idx)

    return selected_indices


def retrieve_chunks(query: str, k: int = 3, use_mmr: bool = True):
    """
    Recupera los k chunks más relevantes para una query.
    Si use_mmr=True, usa MMR para mejorar diversidad de los resultados.
    """
    query_vec = embed_query(query)
    distances, indices = nn_index.kneighbors(query_vec, n_neighbors=10)

    candidate_indices = indices[0]
    candidate_vecs = X_embeddings[candidate_indices]

    if use_mmr:
        selected_local = maximal_marginal_relevance(
            query_vec=query_vec,
            candidate_vecs=candidate_vecs,
            lambda_param=0.7,
            k=k
        )
    else:
        selected_local = list(range(min(k, len(candidate_indices))))

    selected_global = [candidate_indices[i] for i in selected_local]

    # Convertir distancia a similitud
    all_scores = 1 - distances[0]
    scores_dict = {idx: float(score) for idx, score in zip(candidate_indices, all_scores)}

    results = []
    for idx in selected_global:
        row = chunks_df.iloc[idx]
        results.append({
            "score": round(scores_dict[idx], 3),
            "doc_id": row["doc_id"],
            "title": row["title"],
            "category": row["category"],
            "chunk_id": row["chunk_id"],
            "chunk_index": row["chunk_index"],
            "text_chunk": row["text_chunk"]
        })

    results_df = pd.DataFrame(results).sort_values("score", ascending=False)

    # Agregar un ranking por documento
    doc_scores = (
        results_df.groupby(["doc_id", "title", "category"])["score"]
        .mean()
        .reset_index()
        .sort_values("score", ascending=False)
        .rename(columns={"score": "doc_score"})
    )

    return results_df, doc_scores


# Ejemplo de prueba
query_demo = "¿Cómo se gestionan los accesos y credenciales en producción?"
res_chunks_demo, res_docs_demo = retrieve_chunks(query_demo, k=4, use_mmr=True)
res_chunks_demo

,score,doc_id,title,category,chunk_id,chunk_index,text_chunk
0,0.847,1,Política de Seguridad de SISRED,seguridad,1_0,0,La política de seguridad de SISRED establece l...
1,0.449,2,Guía de Despliegue en Producción,despliegue,2_0,0,El despliegue de nuevas versiones de SISRED en...
3,0.259,1,Política de Seguridad de SISRED,seguridad,1_1,1,Se prohíbe explícitamente compartir contraseña...
2,0.232,5,Lineamientos para Proyectos de IA en SISRED,ia,5_1,1,La preparación de datos debe incluir anonimiza...


In [24]:
# ============================================================
# Celda 15 – Ver ranking agregado por documento
# ============================================================

res_docs_demo


,doc_id,title,category,doc_score
0,1,Política de Seguridad de SISRED,seguridad,0.553
1,2,Guía de Despliegue en Producción,despliegue,0.449
2,5,Lineamientos para Proyectos de IA en SISRED,ia,0.232


## 6. Mini pipeline de respuesta (la "G" de RAG)

En un RAG real:

1. Tomamos la **pregunta del usuario**.
2. Recuperamos los chunks más relevantes (lo que ya sabemos hacer).
3. Construimos un **contexto** concatenando esos chunks.
4. Construimos un **prompt** que combina:
   - La pregunta (`query`).
   - El contexto.
5. Enviamos ese prompt a un **LLM** (GPT, Llama, etc.) para que genere la respuesta final.

En este notebook vamos a simular la parte de generación con un enfoque simple:

- Construiremos un contexto consolidado con los chunks recuperados.
- Extraeremos las oraciones más relevantes para la pregunta usando TF-IDF.
- Generaremos una respuesta tipo “resumen ejecutivo” apoyada en ese contexto.

La idea es que se vea claramente **de dónde sale la información**.


In [26]:
# ============================================================
# Celda 17 – Construcción de contexto y respuesta simplificada
# ============================================================

def build_context_from_results(results_df: pd.DataFrame) -> str:
    """
    Une los text_chunk recuperados para formar un contexto consolidado,
    anotando de qué documento viene cada fragmento.
    """
    context_pieces = []
    for _, row in results_df.iterrows():
        header = f"[Doc {row['doc_id']} – {row['title']} | {row['category']}]"
        context_pieces.append(f"{header}\n{row['text_chunk']}")
    return "\n\n".join(context_pieces)


def extractive_summary_from_context(context: str, query: str, max_sentences: int = 4) -> str:
    """
    Resume el contexto seleccionando las oraciones más relevantes para la query.
    Aproximación:
    - Dividir contexto en oraciones.
    - Vectorizar oraciones + query con TF-IDF (local para el resumen).
    - Ordenar oraciones por similitud con la query.
    """
    # Dividir contexto en pseudo-oraciones
    sentences = re.split(r"(?<=[\.\!\?])\s+", context)
    sentences = [s.strip() for s in sentences if s.strip()]

    if not sentences:
        return "No se encontró suficiente contexto para generar un resumen."

    # Vectorizar oraciones + query
    # Usamos la lista existente de stopwords en español (spanish_stopwords) definida previamente.
    local_vectorizer = TfidfVectorizer(stop_words=spanish_stopwords)
    corpus = sentences + [query]
    X_local = local_vectorizer.fit_transform(corpus)

    # La última fila es la query
    query_vec = X_local[-1]
    sent_vecs = X_local[:-1]

    # Calcular similitud coseno (producto punto dado que TF-IDF no está normalizado a 1, pero sirve de aproximación)
    sim_scores = (sent_vecs @ query_vec.T).toarray().ravel()

    # Seleccionar top oraciones
    top_indices = np.argsort(sim_scores)[::-1][:max_sentences]
    selected_sentences = [sentences[i] for i in top_indices]

    # Mantener el orden original para legibilidad
    selected_sentences_sorted = sorted(selected_sentences, key=lambda s: sentences.index(s))

    return " ".join(selected_sentences_sorted)


def simple_rag_answer(query: str, k: int = 4, max_sentences: int = 4):
    """
    Pipeline RAG simplificado:
    1. Recupera k chunks relevantes (con MMR).
    2. Construye un contexto consolidado.
    3. Genera una 'respuesta' extractiva simple.
    """
    res_chunks, res_docs = retrieve_chunks(query, k=k, use_mmr=True)
    context = build_context_from_results(res_chunks)
    summary = extractive_summary_from_context(context, query, max_sentences=max_sentences)

    answer = f"""
    Pregunta del usuario:
    {query}

    Documentos más relevantes (promedio de score por documento):
    {res_docs.to_string(index=False)}

    Resumen basado en el contexto recuperado:
    {summary}

    Nota:
    En un sistema RAG real, este resumen sería reemplazado por la salida de un modelo
    de lenguaje (LLM), usando este mismo contexto como entrada para generar una
    respuesta más detallada, con ejemplos y redacción natural.
    """

    return res_chunks, res_docs, answer


# Probar el pipeline con una pregunta sobre despliegue
query_demo_2 = "¿Qué pasos debo seguir para desplegar una nueva versión de SISRED en producción?"
chunks_ans, docs_ans, answer_text = simple_rag_answer(query_demo_2, k=4, max_sentences=4)

chunks_ans


,score,doc_id,title,category,chunk_id,chunk_index,text_chunk
0,0.905,2,Guía de Despliegue en Producción,despliegue,2_0,0,El despliegue de nuevas versiones de SISRED en...
1,0.364,2,Guía de Despliegue en Producción,despliegue,2_2,2,Los logs de despliegue y monitoreo se almacena...
2,0.215,1,Política de Seguridad de SISRED,seguridad,1_0,0,La política de seguridad de SISRED establece l...
3,0.159,5,Lineamientos para Proyectos de IA en SISRED,ia,5_1,1,La preparación de datos debe incluir anonimiza...


In [27]:
# ============================================================
# Celda 18 – Mostrar la respuesta completa para la query demo
# ============================================================

print(answer_text)



    Pregunta del usuario:
    ¿Qué pasos debo seguir para desplegar una nueva versión de SISRED en producción?

    Documentos más relevantes (promedio de score por documento):
     doc_id                                       title   category  doc_score
      2            Guía de Despliegue en Producción despliegue     0.6345
      1             Política de Seguridad de SISRED  seguridad     0.2150
      5 Lineamientos para Proyectos de IA en SISRED         ia     0.1590

    Resumen basado en el contexto recuperado:
    [Doc 2 – Guía de Despliegue en Producción | despliegue]
El despliegue de nuevas versiones de SISRED en producción debe seguir un flujo de integración continua. Todo despliegue debe quedar trazado con un identificador de versión (tag) y el responsable técnico asociado. [Doc 1 – Política de Seguridad de SISRED | seguridad]
La política de seguridad de SISRED establece lineamientos obligatorios para el manejo de credenciales, control de accesos y protección de datos sens

## 7. Inspeccionar cómo el sistema “ve” una pregunta

Para que el equipo de desarrollo entienda mejor cómo funciona el sistema, es útil:

- Ver qué términos de la query están en el vocabulario TF-IDF.
- Ver su **idf** (qué tan informativos son).
- Entender por qué ciertos documentos o chunks salen arriba.

Vamos a crear una función `inspect_query_terms(query)` que:

1. Tokeniza la query.
2. Muestra qué tokens están en el vocabulario.
3. Entrega el idf aproximado de cada uno (a mayor idf, más informativo).


In [28]:
# ============================================================
# Celda 20 – Inspeccionar términos de la query en el vocabulario TF-IDF
# ============================================================

def inspect_query_terms(query: str, top_n: int = 20):
    tokens = re.findall(r"\b\w+\b", query.lower())
    vocab = vectorizer.vocabulary_
    idf = vectorizer.idf_

    rows = []
    for t in tokens:
        if t in vocab:
            idx = vocab[t]
            rows.append({"term": t, "idf": idf[idx]})
        else:
            rows.append({"term": t, "idf": None})

    df_terms = pd.DataFrame(rows).drop_duplicates(subset=["term"])
    df_terms = df_terms.sort_values("idf", ascending=False, na_position="last")
    return df_terms.head(top_n)


inspect_query_terms("gestión de incidentes críticos en producción y seguridad de accesos")


,term,idf
3,críticos,2.540445
9,accesos,2.540445
0,gestión,2.252763
5,producción,2.252763
7,seguridad,2.252763
2,incidentes,2.029619
1,de,NaN
4,en,NaN
6,y,NaN


## 8. Mini-desafío – Preguntas institucionales con análisis

🎯 **Objetivo del mini-desafío**

Cada participante debe:

1. Proponer **2–3 preguntas** relacionadas con:
   - Políticas de seguridad.
   - Despliegue en producción.
   - Gestión de incidentes.
   - Proyectos de IA y su monitoreo en SISRED.
2. Para cada pregunta:
   - Usar `simple_rag_answer(query, k=4, max_sentences=4)`.
   - Revisar:
     - Los **chunks** recuperados (`res_chunks`).
     - El **ranking de documentos** (`res_docs`).
     - El resumen generado.
3. Responder:
   - ¿Los documentos que aparecen como más relevantes tienen sentido?
   - ¿El resumen realmente responde la pregunta o falta contexto?
   - ¿Qué cambiarías en:
     - La forma de chunkear (tamaño de ventana, solapamiento).
     - Los documentos base.
     - El tipo de pregunta (más específica o más general).

✍️ **Ejemplos de preguntas para inspirarse**:

- "¿Cómo se deben manejar las credenciales de acceso a los sistemas de producción?"
- "¿Qué etapas se consideran en la gestión de un incidente crítico en SISRED?"
- "¿Qué buenas prácticas de desarrollo se recomiendan para nuevas funcionalidades?"
- "¿Qué consideraciones especiales existen al desarrollar proyectos de IA en SISRED?"

> En una implementación productiva, aquí conectaríamos este pipeline con un LLM
> (por ejemplo, GPT) para convertir el contexto en respuestas conversacionales,
> pero la lógica de **chunking + embeddings + índice vectorial** sería esencialmente la misma.


In [29]:
# ============================================================
# Celda 22 – Espacio para tus propias preguntas (ejercicio)
# ============================================================

mis_preguntas = [
    "Escribe aquí tu primera pregunta sobre SISRED o proyectos de IA",
    "Escribe aquí tu segunda pregunta",
    # Agrega más preguntas si quieres...
]

for q in mis_preguntas:
    print("=" * 100)
    print("PREGUNTA:", q)
    res_chunks, res_docs, ans = simple_rag_answer(q, k=4, max_sentences=4)

    print("\n--- Ranking por documento ---")
    display(res_docs)

    print("\n--- Chunks seleccionados ---")
    display(res_chunks[["score", "doc_id", "title", "chunk_index", "text_chunk"]])

    print("\n--- Resumen / Respuesta ---")
    print(ans)


PREGUNTA: Escribe aquí tu primera pregunta sobre SISRED o proyectos de IA

--- Ranking por documento ---


,doc_id,title,category,doc_score
1,5,Lineamientos para Proyectos de IA en SISRED,ia,0.623667
0,3,Procedimiento de Gestión de Incidentes,operaciones,0.118000



--- Chunks seleccionados ---


,score,doc_id,title,chunk_index,text_chunk
0,0.719,5,Lineamientos para Proyectos de IA en SISRED,0,Los proyectos de Inteligencia Artificial en SI...
1,0.705,5,Lineamientos para Proyectos de IA en SISRED,2,"Para modelos de IA generativa, se debe documen..."
2,0.447,5,Lineamientos para Proyectos de IA en SISRED,1,La preparación de datos debe incluir anonimiza...
3,0.118,3,Procedimiento de Gestión de Incidentes,0,La gestión de incidentes en SISRED se divide e...



--- Resumen / Respuesta ---

    Pregunta del usuario:
    Escribe aquí tu primera pregunta sobre SISRED o proyectos de IA

    Documentos más relevantes (promedio de score por documento):
     doc_id                                       title    category  doc_score
      5 Lineamientos para Proyectos de IA en SISRED          ia   0.623667
      3      Procedimiento de Gestión de Incidentes operaciones   0.118000

    Resumen basado en el contexto recuperado:
    [Doc 5 – Lineamientos para Proyectos de IA en SISRED | ia]
Los proyectos de Inteligencia Artificial en SISRED deben alinearse con los objetivos institucionales y respetar principios de transparencia y trazabilidad. [Doc 5 – Lineamientos para Proyectos de IA en SISRED | ia]
Para modelos de IA generativa, se debe documentar el contexto de uso, límites de decisión y mecanismos de supervisión humana. Toda integración de IA con SISRED debe pasar por revisión de seguridad, cumplimiento normativo y pruebas de robustez. [Doc 5 – Lin

,doc_id,title,category,doc_score
0,1,Política de Seguridad de SISRED,seguridad,0.0
1,2,Guía de Despliegue en Producción,despliegue,0.0
2,3,Procedimiento de Gestión de Incidentes,operaciones,0.0
3,4,Buenas Prácticas de Desarrollo en SISRED,desarrollo,0.0



--- Chunks seleccionados ---


,score,doc_id,title,chunk_index,text_chunk
0,0.0,3,Procedimiento de Gestión de Incidentes,2,"Finalizada la mitigación, se ejecuta el anális..."
1,0.0,4,Buenas Prácticas de Desarrollo en SISRED,0,El desarrollo de nuevas funcionalidades para S...
2,0.0,2,Guía de Despliegue en Producción,2,Los logs de despliegue y monitoreo se almacena...
3,0.0,1,Política de Seguridad de SISRED,0,La política de seguridad de SISRED establece l...



--- Resumen / Respuesta ---

    Pregunta del usuario:
    Escribe aquí tu segunda pregunta

    Documentos más relevantes (promedio de score por documento):
     doc_id                                    title    category  doc_score
      1          Política de Seguridad de SISRED   seguridad        0.0
      2         Guía de Despliegue en Producción  despliegue        0.0
      3   Procedimiento de Gestión de Incidentes operaciones        0.0
      4 Buenas Prácticas de Desarrollo en SISRED  desarrollo        0.0

    Resumen basado en el contexto recuperado:
    Todo despliegue debe quedar trazado con un identificador de versión (tag) y el responsable técnico asociado. [Doc 1 – Política de Seguridad de SISRED | seguridad]
La política de seguridad de SISRED establece lineamientos obligatorios para el manejo de credenciales, control de accesos y protección de datos sensibles de los usuarios internos y externos. Todo acceso a sistemas de producción debe realizarse mediante autenticac